In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

Using TensorFlow backend.
C:\Users\arung\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\arung\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\arung\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\arung\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
# Set parameters here 
INPUT_SIZE = 256
mapping = {0:'normal', 1:'other'}
NUMCLASSES = len(mapping)
BATCH_SIZE = 32

folder = r'C:\Users\arung\OneDrive\Desktop\COVID 19 Chest Xray\nih'

trainFolder = folder + r'\train'
valFolder = folder + r'\val'
testFolder = folder + r'\test'


In [3]:
from skimage import exposure

def preprocess(img):
    # Contrast stretching
    p2, p98 = np.percentile(img, (2, 98))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))

In [4]:
train_datagen = ImageDataGenerator(
                                featurewise_center=False,
                                samplewise_center=False,
                                featurewise_std_normalization=False,
                                samplewise_std_normalization=False,
                                zca_whitening=False,
                                zca_epsilon=1e-06,
                                rotation_range=0,
                                width_shift_range=0.0,
                                height_shift_range=0.0,
                                brightness_range=None,
                                shear_range=0.0,
                                zoom_range=0.0,
                                channel_shift_range=0.0,
                                fill_mode="nearest",
                                cval=0.0,
                                horizontal_flip=False,
                                vertical_flip=False,
                                rescale=None,
                                preprocessing_function=preprocess,
                                data_format=None,
                                validation_split=0.2,
                                dtype=None,
                            )
test_datagen = ImageDataGenerator(
                                featurewise_center=False,
                                samplewise_center=False,
                                featurewise_std_normalization=False,
                                samplewise_std_normalization=False,
                                zca_whitening=False,
                                zca_epsilon=1e-06,
                                rotation_range=0,
                                width_shift_range=0.0,
                                height_shift_range=0.0,
                                brightness_range=None,
                                shear_range=0.0,
                                zoom_range=0.0,
                                channel_shift_range=0.0,
                                fill_mode="nearest",
                                cval=0.0,
                                horizontal_flip=False,
                                vertical_flip=False,
                                rescale=None,
                                preprocessing_function=preprocess,
                                data_format=None,
                                validation_split=0.2,
                                dtype=None,
                            )

In [5]:
df = pd.read_csv(folder + r'\allfiles.csv')
df = df[df.label!='pneumonia']
# This next line is to use only 10% of the data
df = df.sample(n = df.shape[0]//10)
print(df['label'].value_counts())

training_set = train_datagen.flow_from_dataframe(
                                            dataframe=df,
                                            directory='',
                                            x_col="filename",
                                            y_col="label",
                                            weight_col=None,
                                            target_size=(INPUT_SIZE, INPUT_SIZE),
                                            color_mode="rgb",
                                            classes=None,
                                            class_mode="categorical",
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            seed=None,
                                            save_to_dir=None,
                                            save_prefix="",
                                            save_format="png",
                                            subset='training',
                                            interpolation="nearest",
                                            validate_filenames=True
                                        )

validation_set = test_datagen.flow_from_dataframe( dataframe=df,
                                            directory='',
                                            x_col="filename",
                                            y_col="label",
                                            weight_col=None,
                                            target_size=(INPUT_SIZE, INPUT_SIZE),
                                            color_mode="rgb",
                                            classes=None,
                                            class_mode="categorical",
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            seed=None,
                                            save_to_dir=None,
                                            save_prefix="",
                                            save_format="png",
                                            subset='validation',
                                            interpolation="nearest",
                                            validate_filenames=True
                                            )

normal    5061
other     3568
Name: label, dtype: int64
Found 6904 validated image filenames belonging to 2 classes.
Found 1725 validated image filenames belonging to 2 classes.


In [6]:
from keras.models import Sequential
import tensorflow as tf
from keras.layers import *
from keras.optimizers import *
#from sklearn.model_selection import train_test_split
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

model = Sequential()

model.add(InputLayer(input_shape=[INPUT_SIZE,INPUT_SIZE,3])) #keras will internally add batch dimension

model.add(Conv2D(filters=32,kernel_size=11,strides=1,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2,padding='same'))
#model.add(Dropout(0.5))

model.add(Conv2D(filters=64,kernel_size=9,strides=1,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2,padding='same'))
#model.add(Dropout(0.5))

model.add(Conv2D(filters=128,kernel_size=7,strides=1,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2,padding='same'))
#model.add(Dropout(0.5))

model.add(Conv2D(filters=256,kernel_size=5,strides=1,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2,padding='same'))
#model.add(Dropout(0.5))

model.add(Conv2D(filters=512,kernel_size=3,strides=1,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2,padding='same'))
#model.add(Dropout(0.5))

model.add(Flatten())


model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(NUMCLASSES,activation='softmax'))


model.compile(optimizer=Adam(lr=0.00002), loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      11648     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      165952    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)       401536    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 256)     

In [ ]:
#Training
#c_weights = {0: 0.4, 1: 0.6}

for epoch in range(0,50):
    print("Epoch",epoch)
    if epoch != 0:
        # Load Model Weights
        model.load_weights('model-normalized-new.h5')    
    history = model.fit_generator(training_set,
    steps_per_epoch=len(training_set),
                   epochs=1,
                   validation_data=validation_set,
                   validation_steps = len(validation_set))

    model.save_weights("model-normalized-new.h5")
    print("Saved model to disk after",epoch+1,"epochs.")


Epoch 0

Epoch 1/1
216/216 [==============================] - 133s 616ms/step - loss: 0.6929 - accuracy: 0.5846 - val_loss: 0.6928 - val_accuracy: 0.5942
Saved model to disk after 1 epochs.
Epoch 1
Epoch 1/1
216/216 [==============================] - 154s 713ms/step - loss: 0.6924 - accuracy: 0.5846 - val_loss: 0.6917 - val_accuracy: 0.5942
Saved model to disk after 2 epochs.
Epoch 2
Epoch 1/1
216/216 [==============================] - 141s 654ms/step - loss: 0.6919 - accuracy: 0.5846 - val_loss: 0.6941 - val_accuracy: 0.5942
Saved model to disk after 3 epochs.
Epoch 3
Epoch 1/1
216/216 [==============================] - 145s 670ms/step - loss: 0.6915 - accuracy: 0.5846 - val_loss: 0.6879 - val_accuracy: 0.5942
Saved model to disk after 4 epochs.
Epoch 4
Epoch 1/1
216/216 [==============================] - 143s 663ms/step - loss: 0.6910 - accuracy: 0.5846 - val_loss: 0.6917 - val_accuracy: 0.5942
Saved model to disk after 5 epochs.
Epoch 5
Epoch 1/1
216/216 [===========================

In [ ]:
history.history.keys()

In [ ]:
fig = plt.figure(figsize=(8,4))
plt.plot(history.history['loss'],color='red')
plt.plot(history.history['acc'],color='green')
plt.plot(history.history['val_loss'],color='magenta')
plt.plot(history.history['val_acc'],color='blue')
plt.show()